## Выбираем лучшие настройки для тикеров

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from app.cache import LocalCache
from bot.vis import TaskProgress
from bot import TestAlgorithm, TestAccAlgorithm
from app.config import RunConfig, AccConfig

test_configs = [
    {
        'acc': AccConfig.from_repr_string('auto_05 [2133193276] |u0.5 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.4(+x0.2)¤'),    
        ]
    },
]

def run_test(config):
    bot_alg_list = []
    for bot_cfg in config['bots']:
        bot_alg_list.append(TestAlgorithm(do_printing=False, config=bot_cfg, use_cache=True))
        
    test_acc_alg = TestAccAlgorithm(
        do_printing=False, 
        config=config['acc'],
        bot_alg_list=bot_alg_list,
        use_cache=True
    )
    
    return test_acc_alg.test(
        last_test_date='2024-10-24',
        test_days_num=7,
    )

# unique_configs = set(test_configs)
unique_configs = test_configs

results = []
LocalCache.clear()
progress = TaskProgress(len(unique_configs))

for cfg in test_configs:
        res = run_test(cfg)
        if res:
            results.append(res)
        progress.update_progress()

print(f"cache {LocalCache.get_counter('cache_miss')} vals, used {LocalCache.get_counter('cache_find')} times")

print(results)

# # Вывод результатов или их дальнейшая обработка
# sorted_results = sorted(results, key=lambda x: (x['config'].ticker, x['config'].pretest_type, -float(x['profit_p'])), reverse=False)
# 
# last_val = ''
# for item in sorted_results:
#     cur_val = item['config'].ticker
#     if cur_val != last_val:
#         last_val = cur_val
#         print()
#     print(item)